# Procesamiento del Lenguaje Natural - TUIA
---
## Trabajo Práctico 1 - Clasificador de Recomendaciones Recreativas utilizando NLP

Integrantes:
- Alsop Agustín (A-4651/7)
- Hachen Rocío (H-1184/3)

### Instalaciones necesarias

In [ ]:
#!python -m spacy download es_core_news_lg

In [ ]:
#!pip install thefuzz python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.7 MB/s eta 0:00:00


In [ ]:
#!pip install transformers sentence_transformers

In [ ]:
#!pip install --upgrade transformers

## Librerias

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from thefuzz import process
import requests
from bs4 import BeautifulSoup
import spacy
from google.colab import output

## Bases de Datos

In [ ]:
# Descargamos los 3 datasets
!gdown 1NhTOYfL1rDCvJkFGbPFGNCALV_NfdlhDSud2-V51v7s

Downloading...
From (original): https://drive.google.com/uc?id=1NhTOYfL1rDCvJkFGbPFGNCALV_NfdlhDSud2-V51v7s
From (redirected): https://docs.google.com/spreadsheets/d/1NhTOYfL1rDCvJkFGbPFGNCALV_NfdlhDSud2-V51v7s/export?format=xlsx
To: /content/NLP data base.xlsx
22.6kB [00:00, 43.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c
To: /content/IMDB-Movie-Data.csv
100% 309k/309k [00:00<00:00, 60.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
To: /content/bgg_database.csv
100% 1.83M/1.83M [00:00<00:00, 42.7MB/s]


### Base de datos para las emociones

In [ ]:
database = '/content/NLP data base.xlsx'
db_emotion = pd.read_excel(database)

In [ ]:
db_emotion['animo'] = db_emotion['animo'].map({
    'triste': -1,
    'neutro': 0,
    'feliz': 1
})

### Bases de datos para recomendar al usuario

In [ ]:
peliculas = '/content/IMDB-Movie-Data.csv'
db_films = pd.read_csv(peliculas)
db_films

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,0.00,45
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,0.00,22


In [ ]:
juego_mesa = '/content/bgg_database.csv'
db_boardgames = pd.read_csv(juego_mesa)
db_boardgames

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.415,8.60,46836.0,Brass: Birmingham is an economic strategy game...,2018,2,4,60,120,14,3.8776,"[{'min': 3, 'max': 4}]","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']","['Hand Management', 'Income', 'Loans', 'Market...","['Age of Reason', 'Economic', 'Industry / Manu..."
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.377,8.53,53807.0,Pandemic Legacy is a co-operative campaign gam...,2015,2,4,60,60,13,2.8308,"[{'min': 4, 'max': 4}]","['Rob Daviau', 'Matt Leacock']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Environmental', 'Medical']"
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.349,8.59,62592.0,Gloomhaven is a game of Euro-inspired tactica...,2017,1,4,60,120,14,3.9132,"[{'min': 3, 'max': 3}]",['Isaac Childres'],"['Action Queue', 'Action Retrieval', 'Campaign...","['Adventure', 'Exploration', 'Fantasy', 'Fight..."
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.335,8.54,44728.0,"In Ark Nova, you will plan and design a modern...",2021,1,4,90,150,14,3.7653,"[{'min': 2, 'max': 2}]",['Mathias Wigge'],"['Action Queue', 'End Game Bonuses', 'Grid Cov...","['Animals', 'Economic', 'Environmental']"
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.240,8.60,24148.0,Twilight Imperium (Fourth Edition) is a game o...,2017,3,6,240,480,14,4.3173,"[{'min': 6, 'max': 6}]","['Dane Beltrami', 'Corey Konieczka', 'Christia...","['Action Drafting', 'Area-Impulse', 'Dice Roll...","['Civilization', 'Economic', 'Exploration', 'N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Tiny Epic Dinosaurs,https://boardgamegeek.com/boardgame/291508/tin...,6.619,7.32,3634.0,"In Tiny Epic Dinosaurs, breakthroughs in moder...",2020,1,4,30,60,14,2.6173,"[{'min': 3, 'max': 3}]",['Scott Almes'],"['Contracts', 'Enclosure', 'Hidden Victory Poi...","['Animals', 'Fantasy', 'Farming', 'Science Fic..."
996,997,Kingsburg (Second Edition),https://boardgamegeek.com/boardgame/199966/kin...,6.618,7.39,3146.0,The realm of Kingsburg is under attack! Monstr...,2017,2,5,90,120,10,2.4512,"[{'min': 4, 'max': 4}]","['Andrea Chiarvesio', 'Luca Iennaco']","['Dice Rolling', 'Tech Trees / Tech Tracks', '...","['City Building', 'Dice', 'Fantasy', 'Medieval']"
997,998,Condottiere,https://boardgamegeek.com/boardgame/112/condot...,6.616,6.88,10260.0,It is 13th century Italy. Trade is flourishing...,1995,2,6,20,45,10,1.9591,"[{'min': 4, 'max': 5}]","['Dominique Ehrhard', 'Duccio Vitale', 'Justin...","['Area Majority / Influence', 'Auction/Bidding...","['Bluffing', 'Card Game', 'Medieval', 'Negotia..."
998,999,Qwirkle,https://boardgamegeek.com/boardgame/25669/qwirkle,6.616,6.77,20334.0,The abstract game of Qwirkle consists of 108 w...,2006,2,4,45,45,6,1.5884,"[{'min': 4, 'max': 4}]",['Susan McKinley Ross'],"['Hand Management', 'Pattern Building', 'Squar...",['Abstract Strategy']


In [ ]:
url = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'
response = requests.get(url)

if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')
    books_list = soup.select('body > div.container > div > ol:nth-child(6) > li > a')

    books = [book.text for book in books_list]

    db_books = pd.DataFrame({'title': books})
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

db_books

,title
0,"Frankenstein; Or, The Modern Prometheus by Mar..."
1,Pride and Prejudice by Jane Austen (2894)
2,The Legend of Sleepy Hollow by Washington Irvi...
3,Romeo and Juliet by William Shakespeare (2506)
4,"Moby Dick; Or, The Whale by Herman Melville (2..."
...,...
995,Relativity : the Special and General Theory by...
996,Areopagitica by John Milton (49)
997,Demian by Hermann Hesse (49)
998,"""The Kingdom of God Is Within You"" by graf Leo..."


### Identificación de frases problemáticas

In [ ]:
nlp = spacy.load("es_core_news_lg")

frases_peligrosas = [
    "suicidio", "suicidarse", "no quiero vivir", "quiero morir", "me voy a matar",
    "pegarse un tiro", "matarme", "acabar con todo", "lastimarme", "me hago daño",
    "me quiero cortar", "terminarlo todo", "me odio", "quiero desaparecer",
    "no puedo seguir así", "me quiero morir", "ojalá desapareciera",
    "me voy de este mundo", "no le importo a nadie", "no tengo a nadie",
    "nadie se preocupa por mí", "ya no aguanto más", "hasta aquí llegué",
    "mi vida no vale nada", "ojalá no despertara", "no tengo miedo a morir",
    "un descanso eterno", "tirar por un puente", "me quiero tirar por un puente"
]

def detectarPatronPeligroso(frase):
    doc_frase = nlp(frase.lower())

    # Verificamos la similitud con las frases peligrosas
    for frase_peligrosa in frases_peligrosas:
        doc_peligroso = nlp(frase_peligrosa)
        if doc_frase.similarity(doc_peligroso) > 0.7:  # Umbral de similitud
            return True

    return False

In [ ]:
def redFlagIdentifier(frase):
  return detectarPatronPeligroso(frase)

In [ ]:
frases_polemicas = [
    "me voy a suicidar",
    "no quiero vivir más",
    "hoy es un mal día",
    "no aguanto esto",
    "me quiero tirar por un puente",
    "la vida no tiene sentido",
    "todos estarían mejor sin mí",
    "me siento vacío",
    "quiero acabar con todo",
    "ojalá desapareciera",
    "me voy de este mundo",
    "espero que esto termine pronto",
    "me siento atrapado",
    "la vida es demasiado difícil",
    "no tengo a nadie que me quiera",
    "voy a matar a alguien"
]

# Iterar sobre las frases polémicas y verificar si se consideran peligrosas
for frase in frases_polemicas:
    resultado = redFlagIdentifier(frase)
    print(f"Frase: '{frase}' - ¿Peligrosa?: {resultado}")

Frase: 'me voy a suicidar' - ¿Peligrosa?: True
Frase: 'no quiero vivir más' - ¿Peligrosa?: True
Frase: 'hoy es un mal día' - ¿Peligrosa?: False
Frase: 'no aguanto esto' - ¿Peligrosa?: True
Frase: 'me quiero tirar por un puente' - ¿Peligrosa?: True
Frase: 'la vida no tiene sentido' - ¿Peligrosa?: False
Frase: 'todos estarían mejor sin mí' - ¿Peligrosa?: True
Frase: 'me siento vacío' - ¿Peligrosa?: True
Frase: 'quiero acabar con todo' - ¿Peligrosa?: True
Frase: 'ojalá desapareciera' - ¿Peligrosa?: True
Frase: 'me voy de este mundo' - ¿Peligrosa?: True
Frase: 'espero que esto termine pronto' - ¿Peligrosa?: True
Frase: 'me siento atrapado' - ¿Peligrosa?: True
Frase: 'la vida es demasiado difícil' - ¿Peligrosa?: False
Frase: 'no tengo a nadie que me quiera' - ¿Peligrosa?: True
Frase: 'voy a matar a alguien' - ¿Peligrosa?: True


## Identificación del estado de ánimo

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
X = db_emotion["frase"].str.lower()
y = db_emotion["animo"].to_numpy() # Convertimos a numpy array

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train.tolist())
X_test_vectorized = model.encode(X_test.tolist())

# Creación y entrenamiento del modelo de Regresión Logística Multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística: 0.7368421052631579
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

          -1       0.79      0.70      0.74        53
           0       0.66      0.74      0.70        47
           1       0.77      0.77      0.77        52

    accuracy                           0.74       152
   macro avg       0.74      0.74      0.74       152
weighted avg       0.74      0.74      0.74       152



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# Nuevas frases para clasificar
palabra = input()
new_phrases = [palabra]

# Preprocesamiento y vectorización de las nuevas frases
new_phrases_lower = [text.lower() for text in new_phrases]
new_phrases_vectorized = model.encode(new_phrases_lower)

# Haciendo predicciones con el modelo entrenado
new_predictions = modelo_LR.predict(new_phrases_vectorized)

# Definimos el mapeo para las etiquetas
labels = {-1: 'triste', 0: 'neutro', 1: 'feliz'}

# Mostrando las predicciones junto con las frases
for text, label in zip(new_phrases, new_predictions):
    print(f"Texto: '{text}'")
    print(f"Clasificación predicha: {labels[label]}\n")

me siento raro
Texto: 'me siento raro'
Clasificación predicha: triste



## Selección de un juego, pelicula o libro

### Fuzzy Search

In [ ]:
# Función para realizar fuzzy search
def fuzzy_search(df, column, query, limit=4, threshold=70):
    return process.extractBests(query, df[column], limit=limit, score_cutoff=threshold)

# Ejemplo de uso
estado_animo = input('Ingrese lo que desea:')
resultados = fuzzy_search(db_films, 'Description', estado_animo)

print(f"Resultados para '{estado_animo}':")
for resultado in resultados:
    Title, score, _ = resultado
    print(f"Title: {Title}, Puntuación: {score}")

""" Ejemplo adicional con una búsqueda más amplia
estado_animo_amplio = 'Garcia'
resultados_amplios = fuzzy_search(df, 'Title', estado_animo_amplio, limit=3, threshold=60)

print(f"\nResultados para '{estado_animo_amplio}' (búsqueda más amplia):")
for resultado in resultados_amplios:
    Title, score, _ = resultado
    print(f"Title: {Title}, Puntuación: {score}")"""

### SBERT

In [ ]:
modelo = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
db_films['tensor'] = db_films['Description'].apply(lambda x: modelo.encode(x, convert_to_tensor=True))

In [ ]:
input_str = input("Ingrese una frase: ")

input_embeddings = modelo.encode(input_str, convert_to_tensor=True)

db_films['coincidencia_sbert'] = db_films['tensor'].apply(lambda x: util.cos_sim(x, input_embeddings).item())

db_films = db_films.sort_values(by='coincidencia_sbert', ascending=False)

db_films[['Title','Description', 'coincidencia_sbert']]

In [ ]:
import time
from IPython.display import clear_output

def loading_animation():
  loading_text = "Espere un momento"
  animation_frames = ["😺", "😸", "😹", "😻", "😼", "😽", "🙀", "😿", "😾"]
  for _ in range(2):
      punto = "."
      for frame in animation_frames:
          punto += "."
          clear_output(wait=True)
          print(f"{loading_text}{punto} \n{frame}")
          time.sleep(0.3)
  print("¡Carga completa!")
  time.sleep(0.3)





In [ ]:
pip install colorama

In [ ]:
from datetime import datetime
import pytz

# Detecta la hora local del usuario en Colab
def saludo():
  local_timezone = pytz.timezone("America/Argentina/Buenos_Aires")
  hora_actual = datetime.now(local_timezone)

  # Determina el saludo basado en la hora
  if 6 <= hora_actual.hour < 12:
      saludo = "¡Buenos días!"
  elif 12 <= hora_actual.hour < 18:
      saludo = "¡Buenas tardes!"
  else:
      saludo = "¡Buenas noches!"

  return hora_actual.hour, saludo


In [ ]:
def menu():
  loading_animation()
  output.clear()

  print('===============================================\n|||||||||||||||||TP 1 - NLP|||||||||||||||||||\n===============================================\n')
  time.sleep(0.3)
  print('Integrantes: ')
  time.sleep(0.3)
  print('Alsop Agustín')
  time.sleep(0.3)
  print('Hachen Rocío')
  time.sleep(1)
  output.clear()

  print(Fore.CYAN + Style.BRIGHT + "=" * 54)
  print(Fore.YELLOW + Style.BRIGHT + "🌟✨  Bienvenido al Sistema de Recomendaciones  ✨🌟")
  print(Fore.CYAN + Style.BRIGHT + "=" * 54)
  time.sleep(1)
  output.clear()
  info_horaria = saludo()
  print(f'{info_horaria[1]} ¿Cómo se siente hoy?')
  user_emotion = [input()]

  # Preprocesamiento y vectorización de las nuevas frases
  new_phrases_lower = [text.lower() for text in user_emotion]
  new_phrases_vectorized = model.encode(new_phrases_lower)

  # Haciendo predicciones con el modelo entrenado
  new_predictions = modelo_LR.predict(new_phrases_vectorized)

  # Definimos el mapeo para las etiquetas
  labels = {-1: 'triste', 0: 'neutro', 1: 'feliz'}

  # Mostrando las predicciones junto con las frases
  for text, label in zip(user_emotion, new_predictions):
      print(f"Texto: '{text}'")
      print(f"Clasificación predicha: {labels[label]}\n")

In [ ]:
menu()

¡Buenas tardes! ¿Cómo se siente hoy?
muy alegre por suerte
Texto: 'muy alegre por suerte'
Clasificación predicha: feliz

